In [ ]:
import sys, importlib
importlib.reload(sys.modules['climatology_ensemble'])

In [7]:
import sys
sys.path.insert(0, "..")

from sf_runoff import daily_climatology, spatial_avg_daily_input, spatial_stats_daily_input, compute_anomalies, create_it_matrix
from nested_CV import SVR_nested_CV_gridsearch, SVR_PCA_nested_CV_gridsearch
from test import evaluate_prediction, plot_prediction, plot_anomalies
from test import nested_CV_PCA_SVR_predict, nested_CV_SVR_predict
from classic_CV_predict import classic_CV_PCA_SVR_predict, classic_CV_SVR_predict


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV,TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA


import matplotlib.pyplot as plt
import os
import pdb
import seaborn as sns

## A) ALL_VARS, T_UNIT = 10

In [8]:
path=r'C:\Users\mmazzolini\OneDrive - Scientific Network South Tyrol\Documents\conda\daily_input\\'

daily_input = pd.read_csv(path+'SAN_BENIGNO_ORCO_2002_2019.csv', index_col=0, parse_dates=True)

daily_input_stat = spatial_stats_daily_input(daily_input)

t_unit=10


#define the possible parameters value (where Gridsearch is applied)

C_range=np.logspace(-1.5, 1.5, 10)
epsilon_range=np.logspace(-5, -2, 5)
#n_range = [17, 50, 200]
components_range = [5*4*36]
#do not enlarge t_range for now
t_range=[36]
t_length=t_range[0]
n_splits=5
test_size=365

In [9]:
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde

from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

import os

import pdb

In [11]:
radius = 0.75

In [21]:

    # This function takes as input the daily temperature, precipitation and runoff and generates the input-target matrix
    runoff = daily_input[['Q']]

    prec = daily_input[[c for c in daily_input.columns if c[0] == 'P']]

    other= daily_input[[c for c in daily_input.columns if (c[0] != 'Q' and c[0] != 'P')]]

    # Compute the t_unit days average runoff
    runoff_t_unit = runoff.rolling(30, min_periods=30).mean()

    
    # Compute the t_unit days average temperature
    if not other.empty:
        other_t_unit = other.rolling(t_unit, min_periods=t_unit).mean()

    # Compute the t_unit days sum precipitation
    if not prec.empty:
        prec_t_unit = prec.rolling(t_unit, min_periods=t_unit).sum()

        
    daily_t_unit = pd.concat([runoff_t_unit ,prec_t_unit, other_t_unit], axis=1)
    daily = daily_t_unit.groupby(by=daily_t_unit.index.day_of_year).mean()
        
    prec_mean=prec_t_unit.groupby(by=prec_t_unit.index.day_of_year).mean()
    prec_q75=prec_t_unit.groupby(by=prec_t_unit.index.day_of_year).quantile(q=0.75)
    prec_q75=prec_q75.add_suffix('_Q75')
    prec_q25=prec_t_unit.groupby(by=prec_t_unit.index.day_of_year).quantile(q=0.25)
    prec_q25=prec_q25.add_suffix('_Q25')

    other_q75=pd.DataFrame(data=None, columns=other_t_unit.columns)
    other_q25=pd.DataFrame(data=None, columns=other_t_unit.columns)


    #get the range as 2/3 of the mean difference between the quantile and the mean precipitation over the year.
    
    range75= radius*(prec_q75.mean().mean()-prec_t_unit).mean().mean()
    range25= radius*(-prec_q25.mean().mean()+prec_t_unit).mean().mean()

    for i in range(1,367):

        #get dates where the dayofyear is ==1 for the 3 variables
        day_i=(prec_t_unit.index.day_of_year==i)
        prec_t_unit_i=prec_t_unit[day_i]
        other_t_unit_i=other_t_unit[day_i]

        #compute the mean and interesting quantiles for the precipitation
        prec_t_unit_i_mean=prec_t_unit_i.mean(axis=1)

        prec_q75_i=prec_q75[prec_q75.index==i].mean(axis=1)
        prec_q25_i=prec_q25[prec_q25.index==i].mean(axis=1)

        #select the situations where the precipitation is similar (closer than a range) to the quantile
        situations75=(np.abs(np.array(prec_t_unit_i_mean)-np.array(prec_q75_i))<range75)
        situations25=(np.abs(np.array(prec_t_unit_i_mean)-np.array(prec_q25_i))<range25)
        

            
        if situations75.sum() != 0:
            if situations75.sum() == 1:
                s=other_t_unit_i[situations75]
                s=pd.DataFrame(data=s.values, index=range(i,i+1),columns=s.columns)
            else:
                s=other_t_unit_i[situations75].mean()
        s.name=i
        other_q75=other_q75.append(s)


        if situations25.sum() != 0:
            if situations25.sum() == 1:
                p=other_t_unit_i[situations25]
                p=pd.DataFrame(data=p.values, index=range(i,i+1),columns=p.columns)
            else:
                p=other_t_unit_i[situations25].mean()
                
        p.name=i
        other_q25=other_q25.append(p)


    other_q75=other_q75.add_suffix('_Q75')
    other_q75.index=list(range(1,367))
    other_q25=other_q25.add_suffix('_Q25')
    other_q25.index=list(range(1,367))
    #pdb.set_trace()
    daily_clim=pd.concat([daily,
                     prec_q75, other_q75,
                     prec_q25, other_q25], axis=1).dropna()


    if daily_clim.index.max() == 365:
        daily_clim.loc[366]=daily_clim.loc[365]


In [22]:
daily_clim

,Q,Px10y0,Px11y0,Px9y1,Px10y1,Px11y1,Px4y2,Px5y2,Px6y2,Px7y2,...,Sx7y5_Q25,Sx8y5_Q25,Sx4y6_Q25,Sx5y6_Q25,Sx6y6_Q25,Sx7y6_Q25,Sx8y6_Q25,Sx5y7_Q25,Sx6y7_Q25,Sx7y7_Q25
1,11.856429,14.060337,15.280095,13.897980,15.453459,16.580920,12.269660,12.010407,12.222529,12.638597,...,68.321429,91.214286,716.521429,215.907143,150.292857,104.664286,97.521429,270.928571,262.692857,151.164286
2,11.463381,13.585154,14.762582,13.474953,14.983575,16.097364,12.183954,11.864882,12.005945,12.372894,...,66.746154,92.000000,755.576923,216.653846,149.038462,102.492308,97.146154,273.069231,264.161538,148.569231
3,11.157405,13.066643,14.128518,13.023103,14.485527,15.418807,12.506110,12.019398,12.164206,12.189833,...,68.776923,89.607692,533.592308,214.253846,146.576923,103.592308,93.938462,274.500000,268.930769,149.253846
4,10.894286,13.178727,14.122795,13.292339,14.646084,15.561560,13.626021,13.032008,12.845628,12.807221,...,77.271429,102.235714,602.171429,233.021429,160.585714,114.864286,106.142857,292.578571,287.421429,164.014286
5,10.647333,11.500612,12.237721,11.534130,12.674558,13.406176,12.914901,12.192360,11.658450,11.458467,...,70.400000,91.453846,539.238462,219.230769,150.538462,105.923077,96.069231,280.407692,274.492308,152.553846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,13.998000,15.154750,16.189498,14.846405,16.529449,17.530219,12.301180,12.152936,12.669832,13.234092,...,61.453846,85.969231,680.461538,215.400000,145.123077,99.853846,92.223077,273.615385,264.484615,146.707692
363,13.475622,15.845736,16.976348,15.357808,17.215654,18.397766,12.748510,12.512302,13.012926,13.581394,...,60.375000,81.100000,704.725000,210.758333,142.883333,96.150000,87.016667,264.716667,259.983333,144.183333
364,12.949422,15.155834,16.254822,14.765064,16.492787,17.660620,12.645558,12.294313,12.751770,13.245945,...,62.292308,87.330769,684.907692,218.861538,147.869231,101.284615,93.807692,278.338462,268.469231,148.376923
365,12.318911,14.914095,16.084103,14.608302,16.299696,17.532204,12.825420,12.509501,12.912985,13.346672,...,64.986667,87.206667,698.966667,218.400000,148.973333,102.593333,94.033333,279.086667,270.746667,152.066667


In [19]:
list(range(1,367))

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [24]:
pd.concat([daily,prec_q75, other_q75,prec_q25, other_q25], axis=1).dropna()

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [31]:
other_q75.loc[~other_q75.index.duplicated(keep='first')]

,Tx10y0_Q75,Tx11y0_Q75,Tx9y1_Q75,Tx10y1_Q75,Tx11y1_Q75,Tx4y2_Q75,Tx5y2_Q75,Tx6y2_Q75,Tx7y2_Q75,Tx8y2_Q75,...,Sx7y5_Q75,Sx8y5_Q75,Sx4y6_Q75,Sx5y6_Q75,Sx6y6_Q75,Sx7y6_Q75,Sx8y6_Q75,Sx5y7_Q75,Sx6y7_Q75,Sx7y7_Q75
1,275.196184,275.226100,275.465313,275.452588,275.259181,272.518789,271.940262,272.386466,274.119216,276.450245,...,126.860000,163.560000,1164.300000,353.600000,262.420000,187.920000,170.840000,407.460000,406.920000,258.480000
2,275.007749,275.037884,275.259188,275.252928,275.066872,272.133257,271.610377,272.093124,273.870785,276.192681,...,121.114286,161.771429,944.857143,355.714286,255.085714,183.214286,165.957143,416.214286,418.928571,252.700000
3,275.211857,275.228608,275.489683,275.467005,275.277508,272.402631,271.893233,272.422206,274.154255,276.454363,...,113.416667,155.350000,1052.800000,346.033333,242.833333,164.016667,160.300000,413.166667,414.366667,233.283333
4,274.341331,274.394707,274.651688,274.688076,274.475288,271.317637,270.865367,271.445729,273.313249,275.579625,...,148.625000,177.925000,948.400000,378.350000,277.000000,201.550000,183.900000,444.750000,453.175000,284.400000
5,275.083182,274.988492,275.524273,275.421669,275.095822,273.108346,272.540661,272.992671,274.508262,276.639668,...,79.300000,109.025000,532.975000,238.425000,164.675000,114.525000,115.575000,286.700000,289.075000,169.650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,276.670461,276.654142,276.883614,276.813833,276.650384,274.109986,273.561717,274.083806,275.568905,277.989571,...,38.300000,45.400000,284.800000,214.600000,117.000000,55.600000,46.100000,294.100000,280.100000,152.100000
363,275.624797,275.595251,275.928095,275.864647,275.597372,273.020084,272.516332,272.998546,274.608192,276.978897,...,76.650000,77.900000,769.200000,197.550000,141.950000,102.750000,86.400000,261.100000,265.350000,165.900000
364,275.701260,275.689591,275.968585,275.917256,275.670335,272.999258,272.515472,272.993980,274.618797,276.969690,...,78.900000,80.550000,772.800000,201.150000,145.300000,105.400000,89.100000,265.150000,269.450000,169.150000
365,275.983977,275.960015,276.264927,276.183780,275.970683,273.562832,272.975179,273.402136,274.969466,277.319749,...,87.300000,125.150000,980.150000,279.133333,195.416667,139.650000,134.500000,344.550000,331.516667,200.150000


In [94]:

        #get dates where the dayofyear is ==1 for the 3 variables
        day_i=(prec_t_unit.index.day_of_year==i)
        prec_t_unit_i=prec_t_unit[day_i]
        other_t_unit_i=other_t_unit[day_i]

        #compute the mean and interesting quantiles for the precipitation
        prec_t_unit_i_mean=prec_t_unit_i.mean(axis=1)

        prec_q75_i=prec_q75[prec_q75.index==i].mean(axis=1)
        prec_q25_i=prec_q25[prec_q25.index==i].mean(axis=1)

        #select the situations where the precipitation is similar (closer than a range) to the quantile
        situations75=(np.abs(np.array(prec_t_unit_i_mean)-np.array(prec_q75_i))<range75)
        situations25=(np.abs(np.array(prec_t_unit_i_mean)-np.array(prec_q25_i))<range25)

        other_q75=other_q75.add_suffix('_Q75')

        other_q25=other_q25.add_suffix('_Q25')

In [108]:
if situations75.sum() != 0:
        if situations75.sum() == 1:
            s=other_t_unit_i[situations75]
            s=pd.DataFrame(data=s.values, index=range(i,i+1),columns=s.columns)
        else:
            s=other_t_unit_i[situations75].mean()
        s.name=i
        other_q75=other_q75.append(s)


In [107]:
pd.DataFrame(data=s.values, index=range(i,i+1),columns=s.columns)

,Tx10y0,Tx11y0,Tx9y1,Tx10y1,Tx11y1,Tx4y2,Tx5y2,Tx6y2,Tx7y2,Tx8y2,...,Sx7y5,Sx8y5,Sx4y6,Sx5y6,Sx6y6,Sx7y6,Sx8y6,Sx5y7,Sx6y7,Sx7y7
190,293.720855,293.716514,293.512796,293.805038,293.844998,286.648385,287.044641,288.581402,291.445754,292.513539,...,0.0,0.0,2468.0,346.5,0.0,0.0,0.0,352.1,279.8,9.9


In [104]:
s.values

array([[2.93720855e+02, 2.93716514e+02, 2.93512796e+02, 2.93805038e+02,
        2.93844998e+02, 2.86648385e+02, 2.87044641e+02, 2.88581402e+02,
        2.91445754e+02, 2.92513539e+02, 2.93203588e+02, 2.93774963e+02,
        2.94126490e+02, 2.79489318e+02, 2.81510857e+02, 2.83835356e+02,
        2.85582171e+02, 2.86719379e+02, 2.88493092e+02, 2.90153839e+02,
        2.91768404e+02, 2.92085166e+02, 2.93593092e+02, 2.94069653e+02,
        2.78395469e+02, 2.79926306e+02, 2.81009910e+02, 2.81926871e+02,
        2.82511944e+02, 2.84285385e+02, 2.86502482e+02, 2.87272276e+02,
        2.87879997e+02, 2.89887423e+02, 2.78124910e+02, 2.79639667e+02,
        2.80219711e+02, 2.79528083e+02, 2.79718350e+02, 2.81481488e+02,
        2.83856843e+02, 2.84530872e+02, 2.84567068e+02, 2.79447034e+02,
        2.80797394e+02, 2.82424551e+02, 2.83353407e+02, 2.84237548e+02,
        2.80687302e+02, 2.80793202e+02, 2.81817359e+02, 4.50190693e+00,
        4.45689220e+00, 4.43121592e+00, 4.52706109e+00, 4.534488

In [102]:
s.index.rename(i)

DatetimeIndex(['2002-07-09'], dtype='datetime64[ns]', name=190, freq=None)

In [88]:
s.reset_index().index.rename(i)

AttributeError: 'RangeIndex' object has no attribute 'reset_index'

In [87]:
s

RangeIndex(start=0, stop=1, step=1, name=190)

In [58]:
s.name = i

In [61]:
s.index = i

TypeError: Index(...) must be called with a collection of some kind, 190 was passed

In [64]:
s.index=pd.index(i)


AttributeError: module 'pandas' has no attribute 'index'

In [74]:
s

Tx10y0    295.284634
Tx11y0    295.256375
Tx9y1     295.120245
Tx10y1    295.411549
Tx11y1    295.410448
             ...    
Sx7y6       0.000000
Sx8y6       0.000000
Sx5y7     528.250000
Sx6y7     463.800000
Sx7y7     113.300000
Name: 192, Length: 153, dtype: float64